In [1]:
using Printf, CSV, DataFrames, Statistics, Glob, XLSX, Dates, Plots, TimeSeries, JSON

In [2]:
directory = joinpath(homedir(), "research", "GADS_Processed", "raw_data")
files = glob("gen_*_type_100_dtgrp_*_rating_*_state_Texas_raw.csv", directory)

133-element Vector{String}:
 "/home/emco4286/research/GADS_Pr" ⋯ 41 bytes ⋯ "_2_rating_1_state_Texas_raw.csv"
 "/home/emco4286/research/GADS_Pr" ⋯ 41 bytes ⋯ "_2_rating_2_state_Texas_raw.csv"
 "/home/emco4286/research/GADS_Pr" ⋯ 41 bytes ⋯ "_4_rating_1_state_Texas_raw.csv"
 "/home/emco4286/research/GADS_Pr" ⋯ 41 bytes ⋯ "_1_rating_2_state_Texas_raw.csv"
 "/home/emco4286/research/GADS_Pr" ⋯ 41 bytes ⋯ "_1_rating_1_state_Texas_raw.csv"
 "/home/emco4286/research/GADS_Pr" ⋯ 41 bytes ⋯ "_3_rating_3_state_Texas_raw.csv"
 "/home/emco4286/research/GADS_Pr" ⋯ 40 bytes ⋯ "_1_rating_3_state_Texas_raw.csv"
 "/home/emco4286/research/GADS_Pr" ⋯ 41 bytes ⋯ "_4_rating_3_state_Texas_raw.csv"
 "/home/emco4286/research/GADS_Pr" ⋯ 41 bytes ⋯ "_1_rating_1_state_Texas_raw.csv"
 "/home/emco4286/research/GADS_Pr" ⋯ 41 bytes ⋯ "_1_rating_1_state_Texas_raw.csv"
 ⋮
 "/home/emco4286/research/GADS_Pr" ⋯ 40 bytes ⋯ "_1_rating_2_state_Texas_raw.csv"
 "/home/emco4286/research/GADS_Pr" ⋯ 40 bytes ⋯ "_1_rating_2_state_

In [3]:
fname = "ercot_hourly_load_2013_2023.csv"
path = joinpath(homedir(), "research", "ERCOT_LOAD_DATA", "processed", fname)
load_data = DataFrame(CSV.File(path))
DataFrames.rename!(load_data, :Hour_End => "x")

load_data = load_data[!, [:x, :ERCOT]]

Row,x,ERCOT
,DateTime,Float64
1,2013-01-01T01:00:00,28684.9
2,2013-01-01T02:00:00,28002.7
3,2013-01-01T03:00:00,27363.8
4,2013-01-01T04:00:00,27032.4
5,2013-01-01T05:00:00,27163.8
6,2013-01-01T06:00:00,27836.8
7,2013-01-01T07:00:00,28722.0
8,2013-01-01T08:00:00,29391.7
9,2013-01-01T09:00:00,30293.8


In [4]:
weather_pt = joinpath(homedir(), "research", "NOAA_WEATHER_DATA", "processed")
weather_files = glob("*.csv", weather_pt)

my_w_list = []

for w in weather_files

    # types_dict=Dict(("Column1" => DateTime))
    weather = DataFrame(CSV.File(w)) #, dateformat="yyyy-mm-dd HH:00:00"))
    weather.Column1 = DateTime.(weather.Column1, "yyyy-mm-dd HH:00:00")
    push!(my_w_list, weather)
end

weather_merged = reduce((x, y) -> innerjoin(x, y, on=:Column1), my_w_list)
DataFrames.rename!(weather_merged, :Column1 => :x)
bool_vec = weather_merged.Tmax .== 0
bool_vec = ifelse.(ismissing.(bool_vec), 0, bool_vec)
bool_vec = convert.(Bool, bool_vec)
idx1 = filter(r -> bool_vec[r], range(1, length(bool_vec)))
weather_merged[idx1, :Tmax] .= missing;


In [5]:
x = Dict()
i = 1
mydict = Dict([("A", 1), ("RS", 1), ("NC", 1),  ("IR", 1), ("MB", 1), ("F", 2), ("U1", 2), ("U2" , 2), ("U3" , 2), ("D1", 2), ("D2", 2), ("D3", 2), ("SF", 2), ("PO", 3), ("PE", 3), ("PD", 3), ("DP", 3), ("MO", 4), ("ME", 4), ("DM", 4), ("D4", 4), ("RU", 5), ("DR", 5), ("TC", 5)])
agedict = Dict([(1, DateTime(1980, 1, 1)), (2, DateTime(1985, 1, 1)), (3, DateTime(1995, 1, 1)), (4, DateTime(2005, 1, 1)), (5, DateTime(2015, 1, 1)), (6, DateTime(2020, 1, 1))])
# agedict = Dict([(2, DateTime(1989, 1, 1)), (3, DateTime(1993, 1, 1))])
types_dict = Dict("EventStartDT" => DateTime, "EventEndDT" => DateTime)
rating_dict = Dict([(1, 50), (2, 150), (3, 200)]);
# weather_zone_dict = Dict([(2, "COAST"), (3, "NORTH_C")]);

Dict{Int64, Int64} with 3 entries:
  2 => 150
  3 => 200
  1 => 50

In [ ]:
# start_date_g = DateTime(2013,1,1,6)
# end_date_g = DateTime(2024,1,1,6)
# date_range_g = [k for k in start_date_g:Hour(1):end_date_g]

# myrange_g = [k for k in 0:1:round(end_date_g - start_date_g, Hour).value]

# f = files[1]

# df = DataFrame(CSV.File(f, types=types_dict))
# df = df[typeof.(df.EventEndDT) .== DateTime, :] # for missing DateTime values
# df = df[df.EventTypeCode .!= "RS", :]

# if size(df)[1] > 1

#     sort!(df, :EventStartDT)
#     transform!(df, :EventTypeCode => ByRow(x -> mydict[x]), :EventTypeCode)

#     birthdate = agedict[df[1, :CommercialDT_grp]]

#     # 
#     state = convert(Array{Union{Missing, Float64}}, fill(1, length(myrange_g))) # to allow missing values
#     cause = convert(Array{Union{Missing, Float64}}, fill(1, length(myrange_g)))
#     age = [k.value for k in round.(date_range_g - birthdate, Day(1))]


#     # Hour that outage starts
#     time_vector = map(x -> round(x - start_date_g, Hour), df[!, :EventStartDT])

#     # Set state to zero during an event
#     for k in [c for c in range(1, step=1, length=length(time_vector))] # k is an index
#         start_time_string = round(df[k, :EventStartDT], Hour)
#         end_time_string = round(df[k, :EventEndDT], Hour)

#         mycause = df[k, :EventTypeCode_function]

#         # if cause == 1 # RS
#         #     continue
#         # end

#         start_hour = time_vector[k].value # integer
#         duration = round(end_time_string - start_time_string, Hour).value
#         end_hour = start_hour + duration

#         start_index = max(1, start_hour)

#         matching_idx = findall(x -> x == end_hour, myrange_g)
        
#         if length(matching_idx) < 1
#             end_index = length(state)
#         else
#             end_index = matching_idx[1]
#         end

#         # If event is unit getting retired, set state as missing until the end of the period and skip to the next unit
#         if cause == 5
#             state[start_index:end] .= missing
#         else
#             state[start_index:end_index] .= 0
#             cause[start_index:end_index] .= mycause
#         end

#     end

#     trajectory = DataFrame(x=date_range_g, y1=state, y2=cause, y3=age)
#     merged = innerjoin(trajectory, load_data[!,[:x, :ERCOT]], on=:x)
#     sort!(merged, :x)
#     merged[!, :y4] = Dates.dayofyear.(merged[!, :x])

#     # Add weather, deal with Tmax = 0
#     merged = innerjoin(merged, weather_merged, on=:x)

#     # Days since last planned outage
#     my_data = Vector{Union{Int,Missing}}(missing, length(merged[!, :x]))

#     indexed = merged[merged.y2 .== 3, :]
#     bool_vec = diff(indexed[!, :x]) .!= Millisecond(3600000)

#     idx1 = filter(r -> bool_vec[r], range(1, length(bool_vec)))
#     event_end_datetimes = indexed[idx1, :x]

#     k = 1
#     for i in merged[!, :x]

#         events_before_i = [k for k in event_end_datetimes if k < i]

#         if length(events_before_i) < 1
#             my_data[1:k] .= missing
#         else


#             val, idx2 = findmin(i .- events_before_i)
#             closest_event = event_end_datetimes[idx2]
#             hours_since_event = round(i - closest_event, Day).value

#             my_data[k] = hours_since_event
#         end

#         k+= 1
#     end

#     merged[!, :y6] = my_data

#     # Days since last maintenance outage
#     my_data = Vector{Union{Int,Missing}}(missing, length(merged[!, :x]))

#     indexed = merged[merged.y2 .== 4, :]
#     bool_vec = diff(indexed[!, :x]) .!= Millisecond(3600000)

#     idx1 = filter(r -> bool_vec[r], range(1, length(bool_vec)))
#     event_end_datetimes = indexed[idx1, :x]

#     k = 1
#     for i in merged[!, :x]

#         events_before_i = [k for k in event_end_datetimes if k < i]

#         if length(events_before_i) < 1
#             my_data[1:k] .= missing
#         else
#             val, idx2 = findmin(i .- events_before_i)
#             closest_event = event_end_datetimes[idx2]
#             hours_since_event = round(i - closest_event, Day).value

#             my_data[k] = hours_since_event
#         end

#         k+= 1
#     end

#     merged[!, :y7] = my_data

#     merged[!, :y8] = map(f -> Month(f).value, merged[!, :x])

#     fname = @sprintf "gen_%d_type_%d_dtgrp_%d_rating_%d_state_Texas_raw.csv" id type dtgrp rating 
#     path = joinpath(homedir(), "research", "GADS_Processed", "trajectories_with_features", fname)
#     CSV.write(path, merged)
    
# end

In [7]:
start_date_g = DateTime(2013,1,1,6)
end_date_g = DateTime(2024,1,1,6)
date_range_g = [k for k in start_date_g:Hour(1):end_date_g]

myrange_g = [k for k in 0:1:round(end_date_g - start_date_g, Hour).value]

for f in files

    @printf "%s \n" f

    mymatch = match(r"/home/emco4286/research/GADS_Processed/raw_data/gen_(\d+)_type_(\d+)_dtgrp_(\d+)_rating_(\d+)_state_Texas_raw.csv", f)
    id = parse(Int32, mymatch[1])
    type = parse(Int32, mymatch[2])
    dtgrp = parse(Int32, mymatch[3])
    rating = parse(Int32, mymatch[4])

    df = DataFrame(CSV.File(f, types=types_dict))
    df = df[typeof.(df.EventEndDT) .== DateTime, :] # for missing DateTime values
    df = df[df.EventTypeCode .!= "RS", :]

    if size(df)[1] > 1

        sort!(df, :EventStartDT)
        transform!(df, :EventTypeCode => ByRow(x -> mydict[x]), :EventTypeCode)

        birthdate = agedict[df[1, :CommercialDT_grp]]

        # 
        state = convert(Array{Union{Missing, Float64}}, fill(1, length(myrange_g))) # to allow missing values
        cause = convert(Array{Union{Missing, Float64}}, fill(1, length(myrange_g)))
        age = [k.value for k in round.(date_range_g - birthdate, Day(1))]


        # Hour that outage starts
        time_vector = map(x -> round(x - start_date_g, Hour), df[!, :EventStartDT])

        # Set state to zero during an event
        for k in [c for c in range(1, step=1, length=length(time_vector))] # k is an index
            start_time_string = round(df[k, :EventStartDT], Hour)
            end_time_string = round(df[k, :EventEndDT], Hour)

            mycause = df[k, :EventTypeCode_function]

            # if cause == 1 # RS
            #     continue
            # end

            start_hour = time_vector[k].value # integer
            duration = round(end_time_string - start_time_string, Hour).value
            end_hour = start_hour + duration

            start_index = max(1, start_hour)

            matching_idx = findall(x -> x == end_hour, myrange_g)
            
            if length(matching_idx) < 1
                end_index = length(state)
            else
                end_index = matching_idx[1]
            end

            # If event is unit getting retired, set state as missing until the end of the period and skip to the next unit
            if mycause == 5
                state[start_index:end] .= missing
                cause[start_index:end] .= missing
            else
                state[start_index:end_index] .= 0
                cause[start_index:end_index] .= mycause
            end

        end

        trajectory = DataFrame(x=date_range_g, y1=state, y2=cause, y3=age)
        mask = map(x -> ismissing(x) ? false : true, trajectory.y2)
        trajectory = trajectory[mask, :]

        merged = innerjoin(trajectory, load_data[!,[:x, :ERCOT]], on=:x)
        sort!(merged, :x)
        merged[!, :y4] = Dates.dayofyear.(merged[!, :x])

        # Add weather, deal with Tmax = 0
        merged = innerjoin(merged, weather_merged, on=:x)

        # Days since last planned outage
        my_data = Vector{Union{Int,Missing}}(missing, length(merged[!, :x]))

        indexed = merged[merged.y2 .== 3, :]
        bool_vec = diff(indexed[!, :x]) .!= Millisecond(3600000)

        idx1 = filter(r -> bool_vec[r], range(1, length(bool_vec)))
        event_end_datetimes = indexed[idx1, :x]

        k = 1
        for i in merged[!, :x]

            events_before_i = [k for k in event_end_datetimes if k < i]

            if length(events_before_i) < 1
                my_data[1:k] .= missing
            else


                val, idx2 = findmin(i .- events_before_i)
                closest_event = event_end_datetimes[idx2]
                hours_since_event = round(i - closest_event, Day).value

                my_data[k] = hours_since_event
            end

            k+= 1
        end

        merged[!, :y6] = my_data

        # Days since last maintenance outage
        my_data = Vector{Union{Int,Missing}}(missing, length(merged[!, :x]))

        indexed = merged[merged.y2 .== 4, :]
        bool_vec = diff(indexed[!, :x]) .!= Millisecond(3600000)

        idx1 = filter(r -> bool_vec[r], range(1, length(bool_vec)))
        event_end_datetimes = indexed[idx1, :x]

        k = 1
        for i in merged[!, :x]

            events_before_i = [k for k in event_end_datetimes if k < i]

            if length(events_before_i) < 1
                my_data[1:k] .= missing
            else
                val, idx2 = findmin(i .- events_before_i)
                closest_event = event_end_datetimes[idx2]
                hours_since_event = round(i - closest_event, Day).value

                my_data[k] = hours_since_event
            end

            k+= 1
        end

        merged[!, :y7] = my_data

        merged[!, :y8] = map(f -> Month(f).value, merged[!, :x])

        fname = @sprintf "gen_%d_type_%d_dtgrp_%d_rating_%d_state_Texas_raw.csv" id type dtgrp rating 
        path = joinpath(homedir(), "research", "GADS_Processed", "trajectories_with_features", fname)
        CSV.write(path, merged)
        
    end

end

/home/emco4286/research/GADS_Processed/raw_data/gen_10267_type_100_dtgrp_2_rating_1_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_10269_type_100_dtgrp_2_rating_2_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_10272_type_100_dtgrp_4_rating_1_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_10295_type_100_dtgrp_1_rating_2_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_10296_type_100_dtgrp_1_rating_1_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_10381_type_100_dtgrp_3_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_1040_type_100_dtgrp_1_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_10510_type_100_dtgrp_4_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_11068_type_100_dtgrp_1_rating_1_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_1106

┌ Warning: thread = 1 warning: error parsing DateTime around row = 42, col = 1: "NULL,", error=INVALID: DELIMITED | INVALID_DELIMITER 
└ @ CSV /home/emco4286/.julia/packages/CSV/XLcqT/src/file.jl:595


/home/emco4286/research/GADS_Processed/raw_data/gen_1660_type_100_dtgrp_1_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_1679_type_100_dtgrp_1_rating_1_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_1680_type_100_dtgrp_1_rating_2_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_1681_type_100_dtgrp_1_rating_2_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_1687_type_100_dtgrp_1_rating_1_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_1688_type_100_dtgrp_1_rating_1_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_1762_type_100_dtgrp_1_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_17_type_100_dtgrp_2_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_2224_type_100_dtgrp_1_rating_1_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_2225_type_100_

┌ Warning: thread = 1 warning: error parsing DateTime around row = 1008, col = 1: "NULL,", error=INVALID: DELIMITED | INVALID_DELIMITER 
└ @ CSV /home/emco4286/.julia/packages/CSV/XLcqT/src/file.jl:595
┌ Warning: thread = 1 warning: error parsing DateTime around row = 1009, col = 1: "NULL,", error=INVALID: DELIMITED | INVALID_DELIMITER 
└ @ CSV /home/emco4286/.julia/packages/CSV/XLcqT/src/file.jl:595


/home/emco4286/research/GADS_Processed/raw_data/gen_5000_type_100_dtgrp_1_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5001_type_100_dtgrp_1_rating_3_state_Texas_raw.csv 


...

/home/emco4286/research/GADS_Processed/raw_data/gen_5027_type_100_dtgrp_1_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5028_type_100_dtgrp_1_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5032_type_100_dtgrp_2_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5048_type_100_dtgrp_1_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5049_type_100_dtgrp_1_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5050_type_100_dtgrp_1_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5051_type_100_dtgrp_1_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5052_type_100_dtgrp_1_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5053_type_100_dtgrp_1_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5054_type_10

┌ Warning: thread = 1 warning: error parsing DateTime around row = 4156, col = 1: "NULL,", error=INVALID: DELIMITED | INVALID_DELIMITER 
└ @ CSV /home/emco4286/.julia/packages/CSV/XLcqT/src/file.jl:595
┌ Warning: thread = 1 warning: error parsing DateTime around row = 4158, col = 1: "NULL,", error=INVALID: DELIMITED | INVALID_DELIMITER 
└ @ CSV /home/emco4286/.julia/packages/CSV/XLcqT/src/file.jl:595


/home/emco4286/research/GADS_Processed/raw_data/gen_5807_type_100_dtgrp_1_rating_2_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5808_type_100_dtgrp_1_rating_2_state_Texas_raw.csv 


┌ Warning: thread = 1 warning: error parsing DateTime around row = 4247, col = 1: "NULL,", error=INVALID: DELIMITED | INVALID_DELIMITER 
└ @ CSV /home/emco4286/.julia/packages/CSV/XLcqT/src/file.jl:595
┌ Warning: thread = 1 warning: error parsing DateTime around row = 4248, col = 1: "NULL,", error=INVALID: DELIMITED | INVALID_DELIMITER 
└ @ CSV /home/emco4286/.julia/packages/CSV/XLcqT/src/file.jl:595


/home/emco4286/research/GADS_Processed/raw_data/gen_5809_type_100_dtgrp_1_rating_2_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5810_type_100_dtgrp_1_rating_2_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5811_type_100_dtgrp_1_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5812_type_100_dtgrp_1_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5814_type_100_dtgrp_1_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5819_type_100_dtgrp_1_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5821_type_100_dtgrp_1_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5822_type_100_dtgrp_1_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5823_type_100_dtgrp_1_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5826_type_10